In [1]:
import string

import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd

nltk.download("punkt", quiet=True)

from sumy.nlp.stemmers import Stemmer
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.utils import get_stop_words

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_colwidth", 1000)


In [2]:
yahoo_df = pd.read_csv(
    "data/yahoo-news-annotated-comments-dataset/ydata-ynacc-v1_0_expert_annotations.tsv",
    sep="\t",
)


In [3]:
yahoo_df.head()


,sdid,commentindex,headline,url,guid,commentid,timestamp,thumbs-up,thumbs-down,text,parentid,constructiveclass,sd_agreement,sd_type,sentiment,tone,commentagreement,topic,intendedaudience,persuasiveness
0,53971,2,Disneyland Worker Found Dead in Haunted Mansion,http://www.cosmopolitan.com/lifestyle/news/a56215/disneyland-paris-haunted-mansion-death/,rjrPtwH5oVVuQnEXX3hf,00003n000000000000000000000000-ed2ae6d0-32ac-471a-b8b2-a718607ee376,1459917444,NaN,NaN,"These things happen , Every job has its dangers.",1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b4249,Constructive,NaN,Positive/respectful,negative,NaN,Disagreement with commenter,Off-topic with article,Reply to a specific commenter,Not persuasive
1,53971,0,Disneyland Worker Found Dead in Haunted Mansion,http://www.cosmopolitan.com/lifestyle/news/a56215/disneyland-paris-haunted-mansion-death/,VaW6HEsuOFUAIBqjw1k~,1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b4249,1459879464,1.0,NaN,"Sad to hear such a bad thing. Very dangerous job working on electricity. One questions though, why did they use a picture the Bates house from Psycho, on a Disney story? Or is that what the Paris Haunted Mansion/Phantom Manor looks like?",NaN,Constructive,NaN,Positive/respectful,mixed,NaN,NaN,Off-topic with article,Broadcast message / general audience,Not persuasive
2,53971,1,Disneyland Worker Found Dead in Haunted Mansion,http://www.cosmopolitan.com/lifestyle/news/a56215/disneyland-paris-haunted-mansion-death/,uwQePj970KaMZuW3~9Q9,00002n000000000000000000000000-1c30b878-b717-4e9a-9872-2ce2906ce783,1459881644,NaN,NaN,Yes..because too many houses in EU look like the original Disney Hunted House so it didn't look scary enough. Bates Motel looks more American and that notion alone scares everyone.,1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b4249,Constructive,NaN,Positive/respectful,neutral,Informative,NaN,Off-topic with article,Reply to a specific commenter,Not persuasive
3,135929,0,This Old Navy Ad Featuring an Interracial Family Is Being Attacked By Racist Trolls,http://mic.com/articles/142323/this-old-navy-ad-featuring-an-interracial-family-is-being-attacked-by-racist-trolls,fixyWJivQjEQtPLLVXsu,1462203719963-3eeffb02-faae-4b51-9174-704c57e6de37,1462203719,18.0,3.0,"I am frankly quite SICK of the phrase ""shoved down our throat"" You know what? Back in the newspaper and three network days you could say that...Now with 300 or more TV channels and an endless internet...You can keep your throat relatively clear of things you don't want...All you have to do is change the channel or click a link to something you DO like... So let's stop it with the ""shoved down our throat"" rhetoric.",NaN,Not constructive,Agreement throughout,Off-topic/digression,negative,Mean,NaN,Off-topic with article,Broadcast message / general audience,Persuasive
4,135929,1,This Old Navy Ad Featuring an Interracial Family Is Being Attacked By Racist Trolls,http://mic.com/articles/142323/this-old-navy-ad-featuring-an-interracial-family-is-being-attacked-by-racist-trolls,_TDnK715vO5y0OzZz_n4,00002I000000000000000000000000-7ef2ac58-bd84-4027-88cf-b865bfe2f1f8,1462204643,7.0,2.0,"Ya, I always wonder why the conservatives are on Yahoo!, whining about all the liberals, when they could be hanging out with their own kind, patting each other on the back, over at Faux News. Don't like what you see, switch the channel. No, they're a bit hypocritical about shoving things down peoples throats, and want to force they're way upon others.",1462203719963-3eeffb02-faae-4b51-9174-704c57e6de37,Not constructive,Agreement throughout,Off-topic/digression,neutral,Sarcastic,Agreement with commenter,Off-topic with article,Reply to a specific commenter,Not persuasive


In [4]:
def get_comment_thread(row: pd.Series) -> str:
    if row["text"][-1] not in string.punctuation:
        row["text"] += "."

    if row["commentindex"] != 0:
        parent_df = yahoo_df[yahoo_df.commentid == row["parentid"]]
        if parent_df.shape[0] == 0:
            return row["text"]
        else:
            return f"{parent_df.iloc[0].thread} {row['text']}"
    else:
        return row["text"]


yahoo_df = yahoo_df.sort_values(by=["commentindex"])
yahoo_df["thread"] = ""
for index, row in yahoo_df.iterrows():
    yahoo_df.at[index, "thread"] = get_comment_thread(row)


In [5]:
LANGUAGE = "english"
NUM_SENTENCES = 3

tokenizer = Tokenizer(LANGUAGE)
stemmer = Stemmer(LANGUAGE)
summarizer = LuhnSummarizer(stemmer)
summarizer.stop_words = get_stop_words(LANGUAGE)


def summarize_thread(row: pd.Series) -> str:
    if not row["thread"]:
        return ""
    parser = PlaintextParser.from_string(row["thread"], tokenizer)
    return "".join([x._text for x in summarizer(parser.document, NUM_SENTENCES)])


yahoo_df["summary"] = yahoo_df.apply(summarize_thread, axis=1)


In [21]:
with pd.option_context(
    "display.max_rows",
    None,
):
    print(yahoo_df.groupby(["sentiment", "commentindex"])["sdid"].count())


sentiment  commentindex
mixed      0                632
           1                431
           2                488
           3                441
           4                394
           5                302
           6                255
           7                184
           8                120
           9                 86
           10                63
           11                50
           12                29
           13                27
           14                12
           15                 6
negative   0               2050
           1               1777
           2               1760
           3               1557
           4               1466
           5               1052
           6                706
           7                488
           8                363
           9                262
           10               193
           11               120
           12                93
           13                41
           14   

In [16]:
with open(
    "data/yahoo-news-annotated-comments-dataset/ydata-ynacc-v1_0_train-ids.txt"
) as f:
    train_ids = [int(x) for x in f.read().splitlines()]

with open(
    "data/yahoo-news-annotated-comments-dataset/ydata-ynacc-v1_0_dev-ids.txt"
) as f:
    dev_ids = [int(x) for x in f.read().splitlines()]

with open(
    "data/yahoo-news-annotated-comments-dataset/ydata-ynacc-v1_0_test-ids.txt"
) as f:
    test_ids = [int(x) for x in f.read().splitlines()]

train_df = yahoo_df[yahoo_df["sdid"].isin(train_ids)]
dev_df = yahoo_df[yahoo_df["sdid"].isin(dev_ids)]
test_df = yahoo_df[yahoo_df["sdid"].isin(test_ids)]


(20477, 22)
(1509, 22)
(1146, 22)


In [40]:
temp_df = yahoo_df.copy(deep=True)
temp_df["train"] = temp_df["sdid"].isin(train_ids)
temp_df["dev"] = temp_df["sdid"].isin(dev_ids)
temp_df["test"] = temp_df["sdid"].isin(test_ids)

with pd.option_context(
    "display.max_rows",
    None,
):
    print(
        temp_df.groupby(
            [
                "sentiment",
                pd.cut(
                    temp_df.commentindex,
                    [0, 3, 6, 9, 12, 15, 18],
                    include_lowest=True,
                    right=False,
                ),
            ]
        )[["train", "dev", "test"]].sum()
    )


                        train  dev  test
sentiment commentindex                  
mixed     [0, 3)         1340  107    86
          [3, 6)         1006   88    35
          [6, 9)          507   27    24
          [9, 12)         189    4     5
          [12, 15)         67    1     0
          [15, 18)          6    0     0
negative  [0, 3)         4764  388   340
          [3, 6)         3541  277   233
          [6, 9)         1407   53    94
          [9, 12)         544   17    13
          [12, 15)        167    0     0
          [15, 18)         11    0     0
neutral   [0, 3)         2278  208   102
          [3, 6)         1873  155    81
          [6, 9)          724   52    24
          [9, 12)         281   12     2
          [12, 15)         90    0     0
          [15, 18)          5    0     0
positive  [0, 3)          685   47    56
          [3, 6)          468   38    32
          [6, 9)          171   15     6
          [9, 12)          48    2     4
          [12, 1